# 7. 텍스트 데이터 다루기
# Working With Text Data

## 7.1 문자열 데이터 타입

데이터 분석을 하면서 문자열 데이터를 접할 수 있는 경우는 아래와 같이 네 가지 종류가 있다.
- 범주형 데이터
- 범주에 의미를 연결시킬 수 있는 임의의 문자열
- 구조화된 문자열 데이터
- 텍스트 데이터

__범주형 데이터__는 고정된 목록(개수)로 구성된다. 예를 들어 가장 좋아하는 색을 묻는 설문에서 `빨강, 녹색, 파랑`과 같이 하나를 선택하는 경우를 말한다.<br />
__범주에 의미를 연결시킬 수 있는 임의의 문자열은__ "사과 같이 빨간색", "불그스름한 색" 은 "빨강"이라는 범주에 연결시킬 수 있는 문자열을 의미한다. <br />
__구조화된 문자열 데이터__는 범주형 데이터 같은 데이터는 아니지만, 주소나 장소, 이메일, 전화번호 등 처럼 일정한 구조를 가진 데이터를 말한다. <br />
__텍스트 데이터__는 뉴스의 기사, 댓글, 논문 등과 같이 일반적인 텍스트 데이터를 말한다. 

텍스트 분석에서 데이터 셋을 ***말뭉치(corpus)***라 하고, 하나의 텍스트를 의미하는 각 데이터 포인트를 ***문서(document)***라고 한다. 

## 7.2 예제 애플리케이션: 영화 리뷰 감성 분석 (Sentiment Analysis)
이 교재에서는 텍스트 분석 예제로 스탠포드 대학교 연구원인 앤드류 마스가 IMDB(Internet Movie DataBase)에서 수집한 영화 리뷰 데이터셋을 사용한다. 해당 데이터 셋은 http://ai.stanford.edu/~amaas/data/sentiment/ 에서 다운 받을 수 있다. 다운로드 후 압축을 풀면 약 400MB 크기의 데이터가 있고 아래와 같이 구성되어 있다.

In [7]:
# Windows10 - 64bit
!tree data/aclImdb

새 볼륨 볼륨에 대한 폴더 경로의 목록입니다.
볼륨 일련 번호가 00000028 CC7F:4282입니다.
D:\USERS\CJH\DEV\STUDY\STUDY\ML_DL\INTRO_TO_ML_WITH_PYTHON\DATA\ACLIMDB
├─test
│  ├─neg
│  └─pos
└─train
    ├─neg
    ├─pos
    └─unsup


`pos`폴더에는 긍정적인(positive) 리뷰가 각각 하나의 파일로 나위어 있고, `neg` 폴더(negative)에도 마찬가지로 있다. `unsup` 폴더는 레이블이 없는 데이터를 담고 있는데, 실습에서는 필요하지 않으므로 삭제해준다. 